In [68]:
import turicreate as tc
import numpy as np

In [69]:
tc.__version__

'6.4.1'

In [70]:
from pymongo import MongoClient

In [71]:
client = MongoClient(serverSelectionTimeoutMS=50)
db = client.turidatabase

In [72]:
db.labeledinstances.find({'dsid': 4})

In [73]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, serverselectiontimeoutms=50), 'turidatabase')

In [74]:
def get_dataset_data(dsid):
    features = []
    labels = []
    
    for a in db.labeledinstances.find({"dsid": dsid}):
        features.append([float(val) for val in a['feature']])
        labels.append(a['label'])
        
    data = {'target': labels, 'sequence':np.array(features)}
    
    return data


def get_dataset_sframe(dsid):
    data = get_dataset_data(dsid)
    return tc.SFrame(data=data)


In [75]:
dsid_4_sframe = get_dataset_sframe(4)

## Create a defualt model

In [76]:
moel = tc.classifier.create(dsid_4_sframe, target='target')

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: The following methods are available for this type of problem.
PROGRESS: BoostedTreesClassifier, RandomForestClassifier, DecisionTreeClassifier, LogisticClassifier
PROGRESS: The returned model will be chosen according to validation accuracy.


Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 1749

Number of classes           : 12

...

| 3         | 6        | 1.000000  | 0.113301     | 0.770154          | 0.731183            |

| 4         | 7        | 1.000000  | 0.144833     | 0.787879          | 0.741935            |

| 9         | 13       | 1.000000  | 0.292386     | 0.861635          | 0.774194            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

PROGRESS: Model selection based on validation accuracy:
PROGRESS: ---------------------------------------------
PROGRESS: BoostedTreesClassifier          : 0.8817204301075269
PROGRESS: RandomForestClassifier          : 0.8924731182795699
PROGRESS: DecisionTreeClassifier          : 0.7311827956989247
PROGRESS: LogisticClassifier              : 0.7741935483870968
PROGRESS: ---------------------------------------------
PROGRESS: Selecting RandomForestClassifier based on validation set performance.


In [77]:
from sklearn.decomposition import PCA

data = get_dataset_data(4)
X = data['sequence']
pca = PCA(n_components=100)
pca.fit(X) # fit data and then transform it
X_pca = pca.transform(X)

data['sequence'] = X_pca
s_frame = tc.SFrame(data=data)

moel = tc.classifier.create(s_frame, target='target')

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: The following methods are available for this type of problem.
PROGRESS: BoostedTreesClassifier, RandomForestClassifier, DecisionTreeClassifier, LogisticClassifier
PROGRESS: The returned model will be chosen according to validation accuracy.


Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 1749

Number of classes           : 12

...

| 3         | 13       | 1.538971  | 0.145295     | 0.882218          | 0.709677            |

| 4         | 14       | 1.538971  | 0.163818     | 0.905660          | 0.688172            |

| 9         | 20       | 1.444827  | 0.267921     | 0.942253          | 0.698925            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

PROGRESS: Model selection based on validation accuracy:
PROGRESS: ---------------------------------------------
PROGRESS: BoostedTreesClassifier          : 0.8064516129032258
PROGRESS: RandomForestClassifier          : 0.8172043010752689
PROGRESS: DecisionTreeClassifier          : 0.6989247311827957
PROGRESS: LogisticClassifier              : 0.6989247311827957
PROGRESS: ---------------------------------------------
PROGRESS: Selecting RandomForestClassifier based on validation set performance.


In [78]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, serverselectiontimeoutms=50), 'turidatabase')

In [79]:
db.trainedmodels.update({"dsid": 1}, {"$set": {"ds_id":1, "ACC:": 0.99, "path": 'path13'}}, upsert=True)

/Users/nicholaslarsen/opt/anaconda3/envs/mlenv2020/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  """Entry point for launching an IPython kernel.


{'n': 1,
 'upserted': ObjectId('618c9dfe14e5094753fd1367'),
 'nModified': 0,
 'ok': 1.0,
 'updatedExisting': False}

In [101]:
doc = db.trainedmodels.find_one({"dsid":4})
doc["count"]

1866

In [102]:
doc["acc_mlp"]

0.9941050375133976

In [103]:
doc

{'_id': ObjectId('618f0adf14e5094753ff8d61'),
 'dsid': 4,
 'acc_mlp': 0.9941050375133976,
 'acc_turi': 0.9914255091103966,
 'count': 1866,
 'path_mlp': '../models/mlp_model_dsid4',
 'path_turi': '../models/mlp_model_dsid4'}

In [100]:
doc["acc_mlp"]

KeyError: 'acc_mlp'